In [9]:
import xlrd
import os 
import sqlite3
import csv

In [28]:
con.close()

In [26]:
db_path = './datawarehouse.db'
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''PRAGMA table_info(weather)'''

cur.execute(query)
con.commit()
r = cur.fetchone()

print(r)

[(0, 'id', 'INT', 0, None, 1), (1, 'weatherdate', 'datetime', 0, None, 0), (2, 'tempmax', 'numeric', 0, None, 0), (3, 'tempmin', 'numeric', 0, None, 0), (4, 'tempday', 'numeric', 0, None, 0), (5, 'feelslikemax', 'numeric', 0, None, 0), (6, 'feelslikemin', 'numeric', 0, None, 0), (7, 'dew', 'numeric', 0, None, 0), (8, 'humidity', 'numeric', 0, None, 0), (9, 'precip', 'numeric', 0, None, 0), (10, 'precipprob', 'numeric', 0, None, 0), (11, 'precipcover', 'numeric', 0, None, 0), (12, 'preciptype', 'numeric', 0, None, 0), (13, 'snow', 'numeric', 0, None, 0), (14, 'snowdepth', 'numeric', 0, None, 0), (15, 'winggust', 'numeric', 0, None, 0), (16, 'windspeed', 'numeric', 0, None, 0), (17, 'winddir', 'numeric', 0, None, 0), (18, 'sealevelpressure', 'numeric', 0, None, 0), (19, 'cloudcover', 'numeric', 0, None, 0), (20, 'visibility', 'numeric', 0, None, 0), (21, 'solarradition', 'numeric', 0, None, 0), (22, 'solarenergy', 'numeric', 0, None, 0), (23, 'uvindex', 'numeric', 0, None, 0), (24, 'seve

In [38]:
db_path = './datawarehouse.db'
con=sqlite3.connect(db_path)
cur = con.cursor()

cur.execute("DROP INDEX IF EXISTS calendar_date_idx")
con.commit()
query = '''CREATE INDEX calendar_date_idx ON calendar(date)'''

cur.execute(query)
con.commit()

cur.execute("DROP INDEX IF EXISTS weather_date_idx")
query = '''CREATE INDEX weather_date_idx ON weather(weatherdate)'''

cur.execute(query)

cur.execute("SELECT * FROM weather INDEXED BY weather_date_idx;")
r = cur.fetchone()
print(r)

con.commit()

con.close()

('2022-12-24', -6.2, -12.8, -9.6, -11, -20.1, -16.3, -19.2, 46, 0, 0, 0, '', 0, 0, 44.1, 25.6, 273.2, 1014.2, 33.2, 16, 102.1, 8.9, 4, 10, '2022-12-24T07:24:38', '2022-12-24T16:51:04', 0.04, 'Partially cloudy', 'Partly cloudy throughout the day.', 'partly-cloudy-day', 'KDCA,72405013743,KADW,KDAA,AS365')


In [39]:
db_path = './datawarehouse.db'
con=sqlite3.connect(db_path)
cur = con.cursor()

cur.execute(''' CREATE TABLE calendar_weather AS
SELECT * 
FROM weather INDEXED BY weather_date_idx 
JOIN calendar INDEXED BY calendar_date_idx 
ON date(weather.weatherdate) = date(calendar.date);''')
r = cur.fetchone()
print(r)
con.close()

None


In [6]:
db_path = './datawarehouse.db'
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT * FROM calendar;'''

cur.execute(query)
print('here')
con.commit()
r = cur.fetchone()

print(r)

con.close()

here
(3686, '2023-03-19', 't', '$67.00', '$67.00', 31, 365)


# Backup

In [10]:
# Full path to xls, including filename
xls_path = './primary/airbnb_data_dict.xls'
airbnb_file = xlrd.open_workbook_xls(xls_path)
airbnb_var_sheets = airbnb_file.sheets()

variable_dict = {}
print('Saved Data Dictonaries:')
for s in airbnb_var_sheets:
    s_name = s.name
    var_stored = True
    
    if s_name.rfind('listings.csv detail v4') != -1:
        variable_dict['listings'] = s
        
    elif s_name.rfind('reviews.csv v1') != -1:
        variable_dict['reviews'] = s
        
    elif s_name.rfind('calendar.csv v2') != -1:
        variable_dict['calendar'] = s
        
    else:
        var_stored = False
        
    if var_stored:
        print(f'\t{s_name}')

Saved Data Dictonaries:
	listings.csv detail v4.3
	listings.csv detail v4
	reviews.csv v1
	calendar.csv v2


In [49]:
# Automatically create the listings table
cur_sheet = variable_dict['listings']
field_vals = cur_sheet.col_values(0)[8:82]
type_vals = cur_sheet.col_values(1)[8:82]

out = f'''-- Table: listings
DROP TABLE IF EXISTS listings;
CREATE TABLE listings(
    id int PRIMARY KEY,\n'''

for val in range (1, len(field_vals)-1):
    var_type = type_vals[val]
    if var_type == 'boolean [t=true; f=false]':
        var_type = 'boolean'
    if var_type == '':
        var_type = 'blob'
    out+=f'''    {field_vals[val]} {var_type},\n'''
    
out+=f'''    {field_vals[len(field_vals)-1]} {var_type}
);\n\n'''

In [50]:
print(len(field_vals))

74


In [45]:
# Automatically create the reviews table
cur_sheet = variable_dict['reviews']
field_vals = cur_sheet.col_values(0)[8:14]
type_vals = cur_sheet.col_values(1)[8:14]

out += f'''-- Table: reviews
DROP TABLE IF EXISTS reviews;
CREATE TABLE reviews(
    id int PRIMARY KEY,
    listing_id int,\n'''

for val in range (1, len(field_vals)-1):
    var_type = type_vals[val]
    if var_type == 'boolean [t=true; f=false]':
        var_type = 'boolean'
    if var_type == '':
        var_type = 'blob'
    out+=f'''    {field_vals[val]} {var_type},\n'''
    
out+=f'''    {field_vals[len(field_vals)-1]} {var_type}
);\n\n'''

In [46]:
# Automatically create the reviews table
cur_sheet = variable_dict['calendar']
field_vals = cur_sheet.col_values(0)[8:15]
type_vals = cur_sheet.col_values(1)[8:15]

out += f'''-- Table: calendar
DROP TABLE IF EXISTS calendar;
CREATE TABLE calendar(
    id int PRIMARY KEY,
    listing_id int,\n'''

for val in range (1, len(field_vals)-1):
    var_type = type_vals[val]
    if var_type == 'boolean [t=true; f=false]':
        var_type = 'boolean'
    if var_type == '':
        var_type = 'blob'
    if field_vals[val]!='id':
        out+=f'''    {field_vals[val]} {var_type},\n'''
    
out+=f'''    {field_vals[len(field_vals)-1]} {var_type}
);\n\n'''

In [47]:
text_file = open("datawarehouse.sql", "w")
n = text_file.write(out)
text_file.close()

In [38]:
field_vals

['listing_id',
 'date',
 'available',
 'price',
 'adjusted_price',
 'minimum_nights',
 'maximum_nights']

In [ ]:


out = f'''-- Table: listings
DROP TABLE IF EXISTS listings;
CREATE TABLE listings(
    id int PRIMARY KEY,\n'''

